In [1]:
import csv
import itertools
import sys
import pandas as pd
PROBS = {
    # Unconditional probabilities for having gene
    "gene": {
        2: 0.01,
        1: 0.03,
        0: 0.96
    },
    "trait": {

        # Probability of trait given two copies of gene
        2: {
            True: 0.65,
            False: 0.35
        },

        # Probability of trait given one copy of gene
        1: {
            True: 0.56,
            False: 0.44
        },

        # Probability of trait given no gene
        0: {
            True: 0.01,
            False: 0.99
        }
    },
    # Mutation probability
    "mutation": 0.01
}
def load_data(filename):
    """
    Load gene and trait data from a file into a dictionary.
    File assumed to be a CSV containing fields name, mother, father, trait.
    mother, father must both be blank, or both be valid names in the CSV.
    trait should be 0 or 1 if trait is known, blank otherwise.
    """
    data = dict()
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row["name"]
            data[name] = {
                "name": name,
                "mother": row["mother"] or None,
                "father": row["father"] or None,
                "trait": (True if row["trait"] == "1" else
                          False if row["trait"] == "0" else None)
            }
    return data
def powerset(s):
    """
    Return a list of all possible subsets of set s.
    """
    s = list(s)
    return [
        set(s) for s in itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s) + 1)
        )
    ]
def joint_probability(people, one_gene, two_genes, have_trait):
    """
    Compute and return a joint probability.

    The probability returned should be the probability that
        * everyone in set `one_gene` has one copy of the gene, and
        * everyone in set `two_genes` has two copies of the gene, and
        * everyone not in `one_gene` or `two_gene` does not have the gene, and
        * everyone in set `have_trait` has the trait, and
        * everyone not in set` have_trait` does not have the trait.
    """
    zero_gene = names-one_gene-two_genes
    prob_table=pd.DataFrame([[1-PROBS["mutation"],PROBS["mutation"]],[0.5*PROBS["mutation"] + 0.5*(1-PROBS["mutation"]),0.5*(1-PROBS["mutation"])+ 0.5*PROBS["mutation"]],[PROBS["mutation"],1-PROBS["mutation"]]],columns=[0,1])
    prob_table=dict(prob_table.T)
    p=1
    count = dict()
    for i in people:
        if i in one_gene:
            count[i]=1
        elif i in two_genes:
            count[i]=2
        else:
            count[i]=0
    for i in people:
        if people[i]['trait']!=None:
            p*=PROBS["gene"][count[i]]*PROBS["trait"][count[i]][people[i]['trait']]
        else:
            mom = people[i]["mother"]
            dad = people[i]["father"]
            mom_count = count[mom]
            dad_count = count[dad]
            child_count = count[i]
            
            if child_count==0: 
                 p*=prob_table[mom_count][0]*prob_table[dad_count][0]
            elif child_count==1:
                 p*=prob_table[mom_count][0]*prob_table[dad_count][1]+prob_table[mom_count][1]*prob_table[dad_count][0]
            else:
                p*=prob_table[mom_count][1]*prob_table[dad_count][1]
            if i in have_trait:
                p*= PROBS["trait"][count[i]][True]
            else:
                p*= PROBS["trait"][count[i]][False]
    return p

def update(probabilities, one_gene, two_genes, have_trait, p):
    """
    Add to `probabilities` a new joint probability `p`.
    Each person should have their "gene" and "trait" distributions updated.
    Which value for each distribution is updated depends on whether
    the person is in `have_gene` and `have_trait`, respectively.
    """
    zero_gene = names-one_gene-two_genes
    count = dict()
    for i in people:
        if i in one_gene:
            count[i]=1
        elif i in two_genes:
            count[i]=2
        else:
            count[i]=0
    for person in people:
        if people[person]['trait']!=None:
                probabilities[person]["gene"][count[person]]+=p
                probabilities[person]["trait"][True]= int(people[person]["trait"])
                probabilities[person]["trait"][False]= 1-int(people[person]["trait"])
        else:
            probabilities[person]["gene"][count[person]]+=p
        if person in have_trait and people[person]['trait']==None:
            probabilities[person]["trait"][True]+=p
        if person not in have_trait and people[person]['trait']==None:
            probabilities[person]["trait"][False]+=p
            
def normalize(probabilities):
    """
    Update `probabilities` such that each probability distribution
    is normalized (i.e., sums to 1, with relative proportions the same).
    """
    for person in probabilities:
        total = sum(list(probabilities[person]["gene"].values()))
        for prob in probabilities[person]["gene"]:
            probabilities[person]["gene"][prob]/=total
        total = sum(list(probabilities[person]["trait"].values()))
        for prob in probabilities[person]["trait"]:
            probabilities[person]["trait"][prob]/=total
    return probabilities

In [2]:
people = load_data("data/family0.csv")

    # Keep track of gene and trait probabilities for each person
probabilities = {
        person: {
            "gene": {
                2: 0,
                1: 0,
                0: 0
            },
            "trait": {
                True: 0,
                False: 0
            }
        }
        for person in people
    }
    # Loop over all sets of people who might have the trait
names = set(people)

    
for have_trait in powerset(names):

        # Check if current set of people violates known information
        fails_evidence = any(
            (people[person]["trait"] is not None and
             people[person]["trait"] != (person in have_trait))
            for person in names
        )
        if fails_evidence:
            continue

        # Loop over all sets of people who might have the gene
        for one_gene in powerset(names):
            for two_genes in powerset(names - one_gene):

                # Update probabilities with new joint probability
                p = joint_probability(people, one_gene, two_genes, have_trait)
                update(probabilities, one_gene, two_genes, have_trait, p)

# Ensure probabilities sum to 1
normalize(probabilities)

    # Print results
for person in people:
    print(f"{person}:")
    for field in probabilities[person]:
        print(f"  {field.capitalize()}:")
        for value in probabilities[person][field]:
            p = probabilities[person][field][value]
            print(f"    {value}: {p:.4f}")

                

Harry:
  Gene:
    2: 0.0092
    1: 0.4557
    0: 0.5351
  Trait:
    True: 0.2665
    False: 0.7335
James:
  Gene:
    2: 0.1976
    1: 0.5106
    0: 0.2918
  Trait:
    True: 1.0000
    False: 0.0000
Lily:
  Gene:
    2: 0.0036
    1: 0.0136
    0: 0.9827
  Trait:
    True: 0.0000
    False: 1.0000
